In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Input, Flatten, Layer, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

In [3]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
! kaggle datasets download amanneo/diabetic-retinopathy-resized-arranged

 99% 1.19G/1.21G [00:11<00:00, 179MB/s]
100% 1.21G/1.21G [00:11<00:00, 110MB/s]


In [ ]:
! unzip diabetic-retinopathy-resized-arranged.zip

###Manual Preprocessing

In [ ]:
def preprocess(file):
    image = Image.open(file)
    image = image.resize((100,100))
    image = np.array(image)
    image = image/255
    return image

In [ ]:
path = "data/"
class_names = os.listdir(path)

image_dataset = []
label_dataset = []


for classname in class_names:
    for image in os.listdir(os.path.join(path,classname)):
        if image.endswith('.jpeg'):
            image = preprocess(os.path.join(path, classname, image))
            image_dataset.append(image)
            label_dataset.append(class_names.index(classname))

### Tensorflow Preprocessing

In [ ]:
image_dataset = "data"
# train_images, test_images = train_test_split(image_dataset, test_size=0.3, random_state=42)
# train_labels = to_categorical(train_labels, 5)
# test_labels = to_categorical(test_labels, 5)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_preprocess = ImageDataGenerator()
test_preprocess = ImageDataGenerator()

In [21]:
train_generator = image_preprocess.flow_from_directory("data",
                                                     target_size=(100, 100),
                                                     batch_size=200,
                                                     class_mode='categorical')

Found 35126 images belonging to 5 classes.


###Old Model

In [ ]:
def layer_architecture():
    input_layer = Input(shape=(100, 100, 3))
    conv_layer1 = Conv2D(64, (10, 10), activation="relu")(input_layer)
    maxpool_layer1 = MaxPooling2D((2, 2), padding="same")(conv_layer1)
    conv_layer2 = Conv2D(128, (7, 7), activation="relu")(maxpool_layer1)
    maxpool_layer2 = MaxPooling2D((2, 2), padding="same")(conv_layer2)
    conv_layer3 = Conv2D(128, (4, 4), activation="relu")(maxpool_layer2)
    maxpool_layer3 = MaxPooling2D((2, 2), padding="same")(conv_layer3)
    conv_layer4 = Conv2D(256, (4, 4), activation="relu")(maxpool_layer3)
    flatten_layer = Flatten()(conv_layer4)
    dense_layer = Dense(5, activation="softmax")(flatten_layer)
    return Model(inputs=input_layer, outputs=dense_layer, name="feature_vector")

In [ ]:
cnn_model = layer_architecture()

In [ ]:
categorical_crossentropy = tf.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
cnn_model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
cnn_model.fit(
    train_images, train_labels,
    epochs=20,
    batch_size=256,
    validation_data=(test_images, test_labels)
)

###New Model

In [11]:
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [13]:
base_model = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 1s 0us/step


In [16]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

In [17]:
model = Model(inputs=base_model.input, outputs=predictions)

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
optimiser = tf.keras.optimizers.Adam(lr=0.001)

In [27]:
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=optimiser, model=model)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_best_only=True,
    monitor='accuracy',
    mode='min',
    verbose=1
)

In [29]:
epochs = 20
steps_per_epoch = train_generator.n // train_generator.batch_size

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    verbose=1,
    callbacks=[checkpoint_callback]
)

Epoch 1/20
175/175 [==============================] - ETA: 0s - loss: 0.6742 - accuracy: 0.7694
Epoch 1: accuracy improved from inf to 0.76937, saving model to ./training_checkpoints/ckpt
175/175 [==============================] - 136s 773ms/step - loss: 0.6742 - accuracy: 0.7694
Epoch 2/20
175/175 [==============================] - ETA: 0s - loss: 0.6434 - accuracy: 0.7758
Epoch 2: accuracy did not improve from 0.76937
175/175 [==============================] - 126s 716ms/step - loss: 0.6434 - accuracy: 0.7758
Epoch 3/20
175/175 [==============================] - ETA: 0s - loss: 0.6089 - accuracy: 0.7874
Epoch 3: accuracy did not improve from 0.76937
175/175 [==============================] - 131s 746ms/step - loss: 0.6089 - accuracy: 0.7874
Epoch 4/20
175/175 [==============================] - ETA: 0s - loss: 0.5655 - accuracy: 0.8025
Epoch 4: accuracy did not improve from 0.76937
175/175 [==============================] - 128s 726ms/step - loss: 0.5655 - accuracy: 0.8025
Epoch 5/20


In [42]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

test_image=load_img("d3.jpg",target_size=(100, 100))
test_image=img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result = model.predict(test_image, verbose=0)

predict={0:"No DR",1:"Mild",2:"Moderate",3:"Severe",4:"Proliferative DR"}

print(predict[np.argmax(result)])

Moderate


In [ ]:
metric1 = Recall()
metric1.update_state(y_true,y_pred)
metric1.result().numpy()

In [ ]:
metric2 = Precision()
metric2.update_state(y_true,y_pred)
metric2.result().numpy()